I want to create a data-driven model to better predict the winners of the current year.
The model i want to create uses a region-based Elo rating. <br/>

We use a genetic algorithm to learn an elo model where the variable are:
- one K constant (positive) for the maximum points gainable/losable per win/lose
- a number of C(Region) costant based on the number of regions. Used in the calculation of the Expected Score.<br/>

Precisely, the Expected Score for A is equal to E(A) = Q(A)/(Q(A)+Q(B)). <br/>
The classic Q(A) function is Q(A) = 10 ^ (R(A)/400), where R(A) represent the Elo rating of the team A. <br/>
In our case, instead, it is Q(A;Region) = 10^(R(A)/C(Region)).<br/>
The Rating is upgraded using the following formula:<br/>
R(A)' = R(A) + K ( S(A) - E(A) ), where the newly introduced S(A) is the score of A. <br/>
This mode is used to calculate the most probable results for the Worlds Main Event. Group and Finals <br/>

I will add another variable called "Elo Degradation" D . This will automatically reduce (or improve, for experimentation) of the elo before the process of upgrading it.<br/>
R(A)' = (R(A)) * D + K ( S(A) - E(A) )<br/>
Starting Elo is 0, this makes for an easier implementation of both Elo scaling and conversion methods.

In [1]:
import mwclient

In [2]:
site = mwclient.Site('lol.gamepedia.com', path='/')

I obtain all of the data I need to create the various worlds dataset

In [3]:
from tqdm.notebook import tqdm
leagues=['GPL','IWCQ','WCS','LCS','LEC','LCK','LPL','CBLOL','LCL','LJL','LLA','OPL','PCS','VCS','TCL','LMS','LST','NA LCS','EU LCS','LLN','CLS']
results = []
for league in tqdm(leagues):
    off=0
    while(True):
        response = site.api('cargoquery',
                            offset=str(off),
                            limit="500",
                            tables = "Tournaments=T, Leagues=L, MatchSchedule=M",
                            fields="L.League_Short=League, T.Name=Tournament, M.DateTime_UTC=Date, M.Team1Final=Team1, M.Team2Final=Team2, M.Team1Score, M.Team2Score",
                            where = 'L.League_Short = "'+league+'"',
                            join_on = "T.League = L.League, T.OverviewPage=M.OverviewPage")
        results += response["cargoquery"] 
        off=off+500
        if(len(response["cargoquery"])<500): break

In [4]:
off=0
while(True):
        response = site.api('cargoquery',
                            offset=str(off),
                            limit="500",
                            tables = "Tournaments=T, Leagues=L, MatchSchedule=M",
                            fields="T.League, T.Name=Tournament, M.DateTime_UTC=Date, M.Team1Final=Team1, M.Team2Final=Team2, M.Team1Score, M.Team2Score",
                            where = 'T.League = "League of Legends SEA Tour"',
                            join_on = "T.League = L.League, T.OverviewPage=M.OverviewPage")
        results += response["cargoquery"] 
        off=off+500
        if(len(response["cargoquery"])<500): break

In [5]:
import pandas as pd
MatchData = pd.DataFrame ( [ a['title'] for a in results ] )

In [6]:
MatchData = MatchData.replace("",float("NaN")).dropna(axis='index', how='any')

Make "date" easier to process

In [7]:
MatchData.sort_values(by="Date",inplace=True)
MatchData = MatchData.astype({'Team1Score':'int32','Team2Score':'int32'})

In [8]:
MatchData["Date"] = pd.to_datetime(MatchData["Date"], format='%Y-%m-%d %H:%M:%S')

In [9]:
MatchData = MatchData.drop("Date__precision", axis=1)

In [10]:
MatchData.League[MatchData.League=='League of Legends SEA Tour']="LST"

E:\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Now we need to take the data we need for the learning process. Precisely we will use the matches from the end of the previous World Championship to the end of the analyzed year World Play-Ins for the learning process.

In [11]:
LearningDates = {
    2020:{'PreviousWCSEnd':'2019-11-11 00:00:00','PlayInEnd':'2020-10-01 00:00:00'},  
    2019:{'PreviousWCSEnd':'2018-11-04 00:00:00','PlayInEnd':'2019-10-09 00:00:00'},
    2018:{'PreviousWCSEnd':'2017-11-05 00:00:00','PlayInEnd':'2018-10-08 00:00:00'},
    2017:{'PreviousWCSEnd':'2016-10-30 00:00:00','PlayInEnd':'2017-09-30 00:00:00'}
}

In [12]:
PerYearMatchData=dict()
for date in LearningDates:
    PerYearMatchData[date] = MatchData[
        (MatchData.Date>LearningDates[date]['PreviousWCSEnd'])&
        (MatchData.Date<LearningDates[date]['PlayInEnd'])]

In [13]:
PerYearMatchData[2017]

,League,Tournament,Date,Team1,Team2,Team1Score,Team2Score
13922,CLS,CLS 2017 Preseason,2016-11-02 21:00:00,Hafnet eSports,Rebirth eSports,2,3
13921,CLS,CLS 2017 Preseason,2016-11-02 21:00:00,Hafnet eSports,Rebirth eSports,2,3
13924,CLS,CLS 2017 Preseason,2016-11-03 21:00:00,Last Kings,Rebirth eSports,3,2
13923,CLS,CLS 2017 Preseason,2016-11-03 21:00:00,Last Kings,Rebirth eSports,3,2
13926,CLS,CLS 2017 Preseason,2016-11-04 21:00:00,Kaos Latin Gamers,Last Kings,0,3
...,...,...,...,...,...,...,...
1309,WCS,Worlds 2017 Play-In,2017-09-26 11:00:00,Hong Kong Attitude,1907 Fenerbahçe Esports,0,1
1310,WCS,Worlds 2017 Play-In,2017-09-28 03:00:00,Cloud9,Lyon Gaming (2013 Latin American Team),3,0
1311,WCS,Worlds 2017 Play-In,2017-09-28 07:00:00,Fnatic,Hong Kong Attitude,3,0
1312,WCS,Worlds 2017 Play-In,2017-09-29 03:00:00,1907 Fenerbahçe Esports,Team oNe eSports,3,1


The leagues appearing in the dataset are

In [14]:
MatchData["League"].unique()

array(['WCS', 'GPL', 'NA LCS', 'EU LCS', 'LPL', 'TCL', 'LJL', 'CBLOL',
       'LMS', 'OPL', 'CLS', 'LCK', 'LCL', 'LLN', 'IWCQ', 'VCS', 'LEC',
       'LLA', 'LCS', 'LST', 'PCS'], dtype=object)

In [15]:
PerYearMatchData[2020]["League"].unique()

array(['VCS', 'LPL', 'LEC', 'CBLOL', 'LCS', 'OPL', 'TCL', 'LCK', 'LJL',
       'LLA', 'LCL', 'PCS', 'WCS'], dtype=object)

We will use the first League a team appears in to decide the "League" it is part of. 
For the compatibility from 2017 to 2020 World Championships we need to alias many of those leagues like this:<br/>
"LCS" <- "NA LCLS"<br/>
"LEC" <- "EU LCS"<br/>
"LCK"<br/>
"LPL"<br/>
"CBLOL"<br/>
"LCL"<br/>
"LJL"<br/>
"LLA" <- "LLN" and "CLS"<br/>
"OPL"<br/>
"PCS" <- "VCS"(was with GPL during 2017), "LMS", "LST" and "GPL"
"TCL" 
       

In [16]:
alias = {
    'GPL': 'PCS', 
    'NA LCS': 'LCS', 
    'EU LCS': 'LEC', 
    'LPL': 'LPL', 
    'TCL': 'PCS', 
    'LJL': 'LJL', 
    'CBLOL': 'CBLOL',
    'LMS': 'PCS', 
    'OPL': 'OPL', 
    'CLS': 'LLA', 
    'LCK': 'LCK', 
    'LCL': 'LCL', 
    'LLN': 'LLA', 
    'VCS': 'PCS', 
    'LEC': 'LEC',
    'LLA': 'LLA', 
    'LCS': 'LCS', 
    'PCS': 'PCS',
    'LST': 'PCS'
}
regions = list(set([alias[name] for name in alias]))



In [17]:
regions_dict = {regions[i]:i for i in range(len(regions))}
regions_indexes = {region:regions_dict[alias[region]] for region in alias}

We should create the Worlds dataset to be used at the end of the calculations to verify our Region-based elo models. Before i didn't take the "Group" column, which can be used to infer the results of the various groups.

In [18]:
WorldsData=dict()
GroupsData=dict()
for year in [2017,2018,2019,2020]:
    response = site.api('cargoquery',
                        limit="max",
                        tables = "MatchSchedule=M",
                        fields="M.Team1Final=Team1, M.Team2Final=Team2, M.Team1Score, M.Team2Score, M.Tab",
                        where = 'M.OverviewPage = "'+str(year)+' Season World Championship/Main Event"')
    WorldsData[year] = pd.DataFrame ( [ a['title'] for a in response['cargoquery'] ] )
    response = site.api('cargoquery',
                        limit="max",
                        tables = "TournamentGroups=T",
                        fields="T.Team, T.GroupName",
                        where = 'T.OverviewPage = "'+str(year)+' Season World Championship/Main Event"')
    GroupsData[year] = pd.DataFrame ( [ a['title'] for a in response['cargoquery'] ] )

In [19]:
GroupsData[2019]

,Team,GroupName
0,ahq eSports Club,Group D
1,Cloud9,Group A
2,Clutch Gaming,Group C
3,DAMWON Gaming,Group D
4,Fnatic,Group C
5,FunPlus Phoenix,Group B
6,G2 Esports,Group A
7,GAM Esports,Group B
8,Griffin,Group A
9,Hong Kong Attitude,Group A


In [20]:
def WhoWon(row):
    if row["Team1Score"]>row["Team2Score"]: return row["Team1"]
    else: return row["Team2"]

In [92]:
FinalPlayOff = dict() # Will contain the matches for the final play-off
for year in [2017,2018,2019,2020]:
    GroupsData[year]["Wins"] = 0
    FinalPlayOff[year] = pd.DataFrame([], columns=WorldsData[year].columns)
    for index,row in WorldsData[year].iterrows():
        if (row.Tab not in ['Quarterfinals', 'Semifinals', 'Finals']):
            winner_row = GroupsData[year].Team == WhoWon(row)
            GroupsData[year].Wins[winner_row] += 1
        else:
            FinalPlayOff[year] = FinalPlayOff[year].append(row)
    FinalPlayOff[year]["Winner"] = FinalPlayOff[year].apply(WhoWon,axis=1)

E:\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
E:\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
E:\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
E:\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to 

In [207]:
import numpy as np
class EloRating:
    def __init__(self,K,C,D,starting_elo=0):
        self.K = max(K,1) # Maximum Elo variation
        self.C = C # Regional Elo correction constant
        self.D = D # Elo degradation constant
        self.start = starting_elo
    def RealElo(self, Rating, Region):
        return Rating/self.C[Region]*400
    def E(self,RatingA, RatingB, RegionA, RegionB):
        return 1/(1+10**(RatingB/self.C[RegionB]-RatingA/self.C[RegionA]))
    def updateRating(self, RatingA, RatingB, RegionA, RegionB, ScoreA):
        E_A = self.E(RatingA, RatingB, RegionA, RegionB)
        return self.D * (RatingA - self.start) + self.K * (ScoreA - E_A)

In [205]:
import copy
class FloatArrayGene:
    def __init__(self, distributions):
        self.dists = distributions
        self.genes = np.array([dist['fun'](*dist['args']) for dist in distributions])

class Mutation:
    def __init__(self): 
        pass
    def mutate(self, gene, **kwargs):
        return gene
    
class MirrorMutation(Mutation):
    def __init__(self, mid): #mid is the middle point
        self.midPoint = mid
    def mutate(self, gene, mp=0.5, di=2): #mp is mutation probability, di a distribution index
        select = np.random.random(len(gene.genes))<mp
        new_gene = copy.deepcopy(gene)
        new_gene.genes[select] = self.midPoint[select] - di*np.random.random()*( gene.genes[select] - self.midPoint[select]) 
        return new_gene
    
class Crossover:
    def __init__(self): 
        pass
    def mutate(self,geneA, geneB, **kwargs):
        return geneA
    
class GeneFlipCrossover(Crossover):
    def __init__(self): 
        pass
    def cross(self, geneA, geneB, cxp=0.5): 
        select = np.random.random(len(geneA.genes))<cxp 
        new_gene_A = copy.deepcopy(geneA)
        new_gene_A.genes[select] = geneB.genes[select]
        return new_gene_A

In [206]:
import numpy as np
# Known K are 26,30(Chess), 258(NFL)
elo_gene_distr = [{'fun':np.random.uniform, 'args': [1,500]}] 
# Known C is 400. Let's move from it, but stay around it
elo_gene_distr += [{'fun':np.random.normal, 'args': [400,80]}]*len(regions) 
# Known D is 1. Let's move from it, but stay around it
elo_gene_distr += [{'fun':np.random.normal, 'args': [1,0.1]}] 

I'm creating the function to evalueate an Elo Model now

In [213]:
from collections import defaultdict
def Score(row):
    return row["Team1Score"]/(row["Team1Score"]+row["Team2Score"])

def WhoWonElo(row):
    if row["Team1Elo"]>row["Team2Elo"]: return row["Team1"]
    else: return row["Team2"]
class EvaluateElo:
    def __init__ (self, matchData, groupData, finalPlayOff, region_indexes):
        self.matches = matchData.sort_values(by="Date")
        self.matches["NormScore"] = self.matches.apply(Score,axis=1)
        self.groups = groupData
        self.playoff = finalPlayOff
        self.region_indexes = region_indexes
    def evaluate(self, EloModel):
        self.Rating = dict()
        self.League = dict()
        #Calculate ELO ratings
        for index, row in self.matches.iterrows():
            if(row.Team1 not in self.League): 
                self.League[row.Team1] = row.League
                self.Rating[row.Team1] = 0
            if(row.Team2 not in self.League): 
                self.League[row.Team2] = row.League
                self.Rating[row.Team2] = 0
            self.Rating[row.Team1] = EloModel.updateRating(self.Rating[row.Team1],
                                                          self.Rating[row.Team2],
                                                          self.region_indexes[self.League[row.Team1]],
                                                          self.region_indexes[self.League[row.Team2]],
                                                          row["NormScore"])
            self.Rating[row.Team2] = EloModel.updateRating(self.Rating[row.Team2],
                                                          self.Rating[row.Team1],
                                                          self.region_indexes[self.League[row.Team2]],
                                                          self.region_indexes[self.League[row.Team1]],
                                                          1-row["NormScore"])
        score = 0
        #Scoring - Group Stage
        Groups = self.groups["GroupName"].unique()
        RealElo = lambda x: EloModel.RealElo(self.Rating[x],self.region_indexes[self.League[x]])
        self.groups["Elo"]=self.groups["Team"].apply(RealElo)
        for group in Groups:
            analyzedGroup = self.groups[self.groups.GroupName==group].sort_values(by="Wins", ascending=False)
            real = analyzedGroup.Team.to_numpy()
            expected = analyzedGroup.sort_values(by="Elo", ascending=False).Team.to_numpy()
            score += np.sum(( real == expected )*[3,3,2,2]) + 2*len(np.intersect1d(real[:2],expected[:2]))
            #Number of people who passed 
        pointValue=5
        self.playoff["Team1Elo"] = self.playoff["Team1"].apply(RealElo)
        self.playoff["Team2Elo"] = self.playoff["Team2"].apply(RealElo)
        self.playoff["RealWinner"] = self.playoff.apply(WhoWon,axis=1)
        self.playoff["ExpectedWinner"] = self.playoff.apply(WhoWonElo,axis=1)
        value = {"Quarterfinals":5,"Semifinals":10,"Finals":20}
        self.playoff["Value"] = self.playoff["Tab"].apply(lambda x: value[x])
        score+=self.playoff["Value"][self.playoff["RealWinner"]==self.playoff["ExpectedWinner"]].sum()
        return score

In [214]:
class LoLEloFitnessEvaluation:
    def __init__ (self, matchData, groupData, finalPlayOff, region_indexes, years):
        self.raters = [ EvaluateElo(matchData[year],
                                    groupData[year],
                                    finalPlayOff[year],
                                    region_indexes) for year in years]
    def evaluate(self, individual):
        eloModel = EloRating(individual.genes[0], individual.genes[1:-1], individual.genes[-1])
        return np.sum([rater.evaluate(eloModel) for rater in self.raters])
        

 Now the the evaluation function is present, we can proceed to write the code for the genetic algorithm. 

In [215]:
class EliteSelection:
    def __init__(self):
        pass
    def select(self, fitnesses, n):
        return np.argsort(fitnesses)[::-1][0:n]

class RouletteWheelSelection:
    def __init__(self):
        pass
    def select(self, fitnesses, n):
        weights = np.divide (fitnesses,np.sum(fitnesses))
        return np.random.choice(np.arange(0,len(fitnesses)),size=n, p=weights)
    
class AdaptiveGeneticAlgorithm:
    def __init__ (self, PopulationGene, SurvivingSelection, ParentsSelection, Mutation, Crossover, FitnessEvaluator):
        self.PopulationGene = PopulationGene
        self.SurvivingSelection = SurvivingSelection
        self.ParentsSelection = ParentsSelection
        self.Mutation = Mutation
        self.Crossover = Crossover
        self.Fitness = FitnessEvaluator
    def evolve(self, total_population, generations=10, adaptiveMutationCrossover=True, startingMutationProb = 0.5, startingCrossoverProb = 1.0):
        #Initialize individuals
        individuals = np.array([self.PopulationGene() for i in range(total_population)])
        mutation = startingMutationProb
        crossover = startingCrossoverProb
        #Calculate fitnesses
        print("evaluating Starting Pop fitness")
        fitnesses = np.array([self.Fitness.evaluate(individual) for individual in tqdm(individuals)])
        order = np.argsort(fitnesses)[::-1]
        fitnesses = fitnesses[order]
        individuals = individuals[order]
        self.best_individual = individuals[0]
        self.best_fitness = fitnesses[0]
        fitness_historical = np.array(self.best_fitness)
        n_gen=0
        while(n_gen<generations):
            print("Generation "+str(n_gen)+" Best Fitness "+str(self.best_fitness))
            new_individuals_mutation = int(total_population*mutation)
            new_individuals_crossover = int(total_population*crossover)
            new_individuals = np.zeros(new_individuals_crossover+new_individuals_mutation, dtype=np.object)
            parents = self.ParentsSelection.select(fitnesses, new_individuals_mutation+2*new_individuals_crossover)
            #Create individuals using Mutation
            new_individuals[0:new_individuals_mutation] = [
                self.Mutation.mutate(individuals[parents[i]]) for i in range(new_individuals_mutation)]
            #Create individuals using Crossover
            new_individuals[new_individuals_mutation:] = [
                self.Crossover.cross(individuals[parents[new_individuals_mutation+2*i]],
                                     individuals[parents[new_individuals_mutation+2*i+1]]) for i in range(new_individuals_crossover)]
            #Calculate fitnesses
            print("evaluating New Pop fitness")
            new_fitnesses = np.array([self.Fitness.evaluate(individual) for individual in tqdm(new_individuals)])
            #Select survivors
            individuals = np.append(individuals, new_individuals)
            fitnesses = np.append(fitnesses, new_fitnesses)
            select = self.SurvivingSelection.select(fitnesses, total_population)
            fitnesses = fitnesses[select]
            individuals = individuals[select]
            if(fitnesses[0]>self.best_fitness):
                
                self.best_individual = individuals[0]
                self.best_fitness = fitnesses[0]
            fitness_historical=np.append(fitness_historical, self.best_fitness)
            if(adaptiveMutationCrossover):
                non_stagnation = self.best_fitness/np.mean(fitness_historical[::-1][0:5])
                mutation = startingMutationProb + non_stagnation*(startingCrossoverProb-startingMutationProb)
                crossover = startingCrossoverProb - non_stagnation*(startingCrossoverProb-startingMutationProb)
            n_gen+=1
        

In [216]:
aga = AdaptiveGeneticAlgorithm(
    lambda : FloatArrayGene(elo_gene_distr),
    EliteSelection(), #Selecting 
    RouletteWheelSelection(), 
    MirrorMutation(np.array([50]+[400]*len(regions)+[1])),
    GeneFlipCrossover(),
    LoLEloFitnessEvaluation(PerYearMatchData, GroupsData, FinalPlayOff, regions_indexes, [2017,2018])
)

In [217]:
aga.evolve(20, generations=20)

evaluating Starting Pop fitness


E:\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()



Generation 0 Best Fitness 124
evaluating New Pop fitness



Generation 1 Best Fitness 146
evaluating New Pop fitness



Generation 2 Best Fitness 146
evaluating New Pop fitness


...


Generation 15 Best Fitness 162
evaluating New Pop fitness



Generation 16 Best Fitness 164
evaluating New Pop fitness



Generation 17 Best Fitness 164
evaluating New Pop fitness



Generation 18 Best Fitness 164
evaluating New Pop fitness



Generation 19 Best Fitness 164
evaluating New Pop fitness


In [218]:
individual=aga.best_individual

In [219]:
Evaluate = LoLEloFitnessEvaluation(PerYearMatchData, GroupsData, FinalPlayOff, regions_indexes, [2017,2018])
Evaluate.evaluate(individual)

164

In [220]:
Evaluate = LoLEloFitnessEvaluation(PerYearMatchData, GroupsData, FinalPlayOff, regions_indexes, [2019])

In [221]:
Evaluate.evaluate(individual)

69

In [222]:
Evaluate = LoLEloFitnessEvaluation(PerYearMatchData, GroupsData, FinalPlayOff, regions_indexes, [2020])

In [223]:
Evaluate.evaluate(individual)

KeyError: 'TBD'

The important part for now is having the following table, to decide the various winners

In [224]:
GroupsData[2020].sort_values(by=["GroupName","New_Elo"], ascending=False)

,Team,GroupName,Wins,Elo,New_Elo
15,Unicorns Of Love.CIS,Group D,3,20.847338,456644.948626
14,Top Esports,Group D,3,11.615048,156935.841438
1,DRX,Group D,3,14.620715,117824.804064
2,FlyQuest,Group D,3,5.044691,68648.709963
5,Gen.G,Group C,3,37.674071,264638.755467
3,Fnatic,Group C,3,9.759814,241353.636043
7,LGD Gaming,Group C,3,-2.128985,81295.050685
13,Team SoloMid,Group C,3,3.683838,53500.270454
9,PSG Talon,Group B,3,6.538271,241318.953616
6,JD Gaming,Group B,3,14.916648,193132.093002


In [146]:
eloModel = EloRating(individual.genes[0], individual.genes[1:-1], individual.genes[-1])

In [179]:
eloModel.D

1.2977749435440376

In [242]:
GroupsData[2019].sort_values(by=["GroupName","Elo"], ascending=False)

,Team,GroupName,Wins,Elo
3,DAMWON Gaming,Group D,5,24.621485
15,Team Liquid,Group D,3,14.566014
10,Invictus Gaming,Group D,4,9.157000
0,ahq eSports Club,Group D,0,1.317168
13,SK Telecom T1,Group C,5,36.906351
4,Fnatic,Group C,4,14.061228
12,Royal Never Give Up,Group C,3,8.691843
2,Clutch Gaming,Group C,0,-5.129687
5,FunPlus Phoenix,Group B,5,18.240158
14,Splyce,Group B,4,11.308333


In [229]:
GroupsData[2020].sort_values(by=["GroupName","Elo"], ascending=False)

,Team,GroupName,Wins,Elo
15,Unicorns Of Love.CIS,Group D,3,20.847338
1,DRX,Group D,3,14.620715
14,Top Esports,Group D,3,11.615048
2,FlyQuest,Group D,3,5.044691
5,Gen.G,Group C,3,37.674071
3,Fnatic,Group C,3,9.759814
13,Team SoloMid,Group C,3,3.683838
7,LGD Gaming,Group C,3,-2.128985
0,DAMWON Gaming,Group B,3,28.740478
6,JD Gaming,Group B,3,14.916648


In [233]:
{region: eloModel.C[regions_indexes[region]] for region in regions_indexes}

{'GPL': 406.16299466788485,
 'NA LCS': 233.30982334646853,
 'EU LCS': 434.58356478234083,
 'LPL': 353.9935943546179,
 'TCL': 406.16299466788485,
 'LJL': 397.16969761929926,
 'CBLOL': 377.5353853763625,
 'LMS': 406.16299466788485,
 'OPL': 309.1793320497195,
 'CLS': 373.15318646185773,
 'LCK': 369.6985027610664,
 'LCL': 303.35532716298246,
 'LLN': 373.15318646185773,
 'VCS': 406.16299466788485,
 'LEC': 434.58356478234083,
 'LLA': 373.15318646185773,
 'LCS': 233.30982334646853,
 'PCS': 406.16299466788485,
 'LST': 406.16299466788485}

In [263]:
def addStandings(df):
    df["Rank"] = 0
    for group in df["GroupName"]:
        ranking = df[df["GroupName"] == group].sort_values(by=["Elo"], ascending=False).index
        df["Rank"].loc[ranking]=np.arange(1,5) 
    return df.sort_values(by=["GroupName","Rank"])

In [261]:
addStandings(GroupsData[2020])

E:\Python\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Team,GroupName,Wins,Elo,Rank
0,DAMWON Gaming,Group B,3,28.740478,1
1,DRX,Group D,3,14.620715,2
2,FlyQuest,Group D,3,5.044691,4
3,Fnatic,Group C,3,9.759814,2
4,G2 Esports,Group A,3,19.587505,1
5,Gen.G,Group C,3,37.674071,1
6,JD Gaming,Group B,3,14.916648,2
7,LGD Gaming,Group C,3,-2.128985,4
8,Machi Esports,Group A,3,13.593080,2
9,PSG Talon,Group B,3,6.538271,4


In [264]:
addStandings(GroupsData[2019])

E:\Python\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Team,GroupName,Wins,Elo,Rank
8,Griffin,Group A,6,37.097126,1
6,G2 Esports,Group A,5,21.481291,2
1,Cloud9,Group A,2,11.442398,3
9,Hong Kong Attitude,Group A,0,3.516345,4
5,FunPlus Phoenix,Group B,5,18.240158,1
14,Splyce,Group B,4,11.308333,2
11,J Team,Group B,3,5.652989,3
7,GAM Esports,Group B,1,2.018356,4
13,SK Telecom T1,Group C,5,36.906351,1
4,Fnatic,Group C,4,14.061228,2
